In [ ]:
options(jupyter.rich_display = F)

# LAB 07: MATRICES

## QUESTION 1

For this question, we will use a pre-built dataset which contains the distances in km's between 81 province centers in Turkey.

If you are running a local version of R-studio, you first need to download the **distance2.rdata** file from the files directory in Binder and open it. In order to download it, you need to open the files directory, check the box on the left hand side of **distance2.rdata** and click download. When you open the rdata file in your computer you will be able to work with the dataset.

If you are running R-studio through Binder, first open the Matrices lab. Then click on the URL on your browser and replace "tree" with "rstudio". It might take a few moments to load. After it's loaded, copy and paste the line below to your console: 

In [ ]:
load("~/file/distance2.RData")

Now write a function named **neighbour** that takes three arguments:

- city: Either a city code as integer (i.e. 34) or city name (i.e. "İstanbul")
- radius: Radius in km within which we want to filter the neighbours of the city
- dist: A distance matrix, default value of which is the **distance2** global object

as such:

```R
neighbour(34, 150)

  yalova  kocaeli    bursa tekirdag  sakarya  bilecik 
      47       78       91      123      127      128 
```

```R
neighbour(city = "Adana", radius = 100)

  mersin osmaniye 
      64       82 

neighbour("İzmir", 50)

manisa 
    33 
```

*Hint: Be careful not to return the same city as its neighbor.*


## QUESTION 2

Now using the same **distance2** object, write a function named **neigh_count** that takes two arguments:

- radius
- dist with a default value of **distance2** object

The function should return the count of neighbouring cities of each city within the specified radius, sorted in descending order.

*Hint 1: Use the ``apply()`` function.*

*Hint 2: Just like the previous question, number of neighbors must not include the cities itselves.*

## QUESTION 3

You are supposed to write a function named **square_donut** that takes two arguments n and x and does the following:

- It should create a square matrix of size n.
- The outer x squares of the matrix should have 1, while the square "hole" in the middle should be all zeros.
- If x is greater than or equal to the half of n, it should throw a message as "edge too wide" and stop.

For example:
```R
square_donut(7, 2)
```
returns
```
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	0	0	0	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1
```
and

```R
square_donut(6, 3)
```
returns
```
'edge too wide'
```

## QUESTION 4

First, set the seed to 1.

Now, rrite a function called **new_field** that takes two inputs, **size** and **density** and creates a square matrix of the given size. The created matrix represents a minefield. The cells should take only two values: if the value is 1, it means there is a mine there and If the value is 0, it means there is no mine. The default values of **size** and **density** should be 5 and 0.2 respectively.

The function should fill the matrix according to **density** which represents the proportion of the cells with mines to the total number of cells. When filling the cells, you should use **density** as the probability of a given cell containing a mine.

Your function should give the following output:

```R
field1 <- new_field()
field1
```
```
0    1    0    0    1
0    1    0    0    0
0    0    0    1    0
1    0    0    0    0
0    0    0    0    0

```

*Hint: You can use the ``sample()`` function when filling the cells.*

## QUESTION 5

Now write a function called **minefield** that takes 3 arguments:

- rw: An integer for row index
- cl: An integer for column index
- field: A matrix for the minefield. Default value is the field1 global object

The function should first retrieve the value of the cell at the rw and cl indices.

If the value is equal to 1, there is a hit, and the function should return 9 (that represents a hit)
Else, there is a miss, then it should return the number of neighbouring hits. In other words, it should return the count of 1's in the neighbouring eight cells of the target.

Some example outputs are given below:

```R
minefield(1,1)
[1] 2

minefield(2,2)
[1] 9

minefield(2,3)
[1] 3
```

## QUESTION 6

You are given a matrix generator function as such:

```R
matrix_gen <- function(maxx = 100, nr = 5, nc = 5)
{
    matrix(sample(maxx, nr * nc, replace = T), nrow = nr) 
}
```

Then write a function called **max_path** that will take a matrix **matt** as input and return the maximum sum to be attained when a path is selected from the first row to the end, in each step of which you can select any of the adjacent cells (straight down, down left, down right) on the next row.

First set the seed to 100.

Now let's try our function with a 3x3 matrix whose cells can take values from 1 to 10. We can construct this matrix using the provided **matrix_gen** function:

```R
mat1 <- matrix_gen(10, 3, 3)
mat1
     [,1] [,2] [,3]
[1,]    4    1    9
[2,]    3    5    4
[3,]    6    5    6
```
```R
max_path(mat1)
[1] 20
```

Let's try it one more time (you need to set the seed to 100 again):

```R
mat2 <- matrix_gen(10, 5, 5)
mat2
     [,1] [,2] [,3] [,4] [,5]
[1,]    4    5    7    7    6
[2,]    3    9    9    3    8
[3,]    6    4    3    4    6
[4,]    1    6    4    4    8
[5,]    5    2    8    7    5
```

```R
max_path(mat2)
[1] 36
```

*Hint: You may make use of ``pmax()`` or ``apply()`` functions.*